In [ ]:
import ipywidgets as widgets
import textwrap
from IPython.display import display, Markdown

##############################################################################
# 계산 로직
##############################################################################

def globalist_value_next_year(
    # 올해
    this_year_organic_nights,
    required_nights_for_globalist,
    mattress_run_cost_per_night,

    # 내년
    next_year_organic_nights,
    next_year_globalist_per_night_value,

    # 포인트 적립
    base_points_per_dollar,
    globalist_bonus_percent,
    point_value_per_point,

    # 마일스톤(올해 달성 시 => 내년에 사용)
    cat14_value,
    cat17_value,
    goh_value,
    club_access_value,
    suite_upgrade_value,

    milestone_30_choice,
    milestone_40_choice,
    milestone_50_choice
):
    """
    올해 매트리스런을 통해 Globalist(60박) 달성 후,
    내년에 얻을 혜택(= 내년 숙박 × Globalist 1박 가치 + 올해 획득 마일스톤 리워드 가치)을
    매트리스런 비용과 비교하는 함수.

    반환:
      {
        "Scenario_A": {...},  # 매트런X, 내년 글로벌리스트X
        "Scenario_B": {...}   # 매트런O, 내년 글로벌리스트O
      }
    """

    ##########################
    # 시나리오 A: 매트리스런 안 함
    ##########################
    scenario_a_additional_cost = 0.0
    scenario_a_additional_value_next_year = 0.0  # 글로벌리스트X → 혜택 0
    scenario_a_net = scenario_a_additional_value_next_year - scenario_a_additional_cost

    ##########################
    # 시나리오 B: 매트리스런으로 올해 60박 달성
    ##########################
    # (1) 올해 부족 박수
    nights_needed = required_nights_for_globalist - this_year_organic_nights
    if nights_needed < 0:
        nights_needed = 0  # 이미 60박 넘어도 굳이 0 처리

    # (2) 매트리스런 총 비용
    scenario_b_mattress_run_cost = nights_needed * mattress_run_cost_per_night

    # (3) 올해 총 박수
    total_nights_year1 = this_year_organic_nights + nights_needed

    # (4) 올해 마일스톤 리워드(내년 사용)
    milestone_value_for_next_year = 0.0

    # 30박: Cat1-4 Free Night + [2K Points or 2 Club Access]
    if total_nights_year1 >= 30:
        milestone_value_for_next_year += cat14_value
        if milestone_30_choice == "2K Points":
            milestone_value_for_next_year += 2000 * point_value_per_point
        else:
            milestone_value_for_next_year += 2 * club_access_value

    # 40박: 1 GOH + [5K Points or 1 Suite Upgrade]
    if total_nights_year1 >= 40:
        milestone_value_for_next_year += goh_value
        if milestone_40_choice == "5K Points":
            milestone_value_for_next_year += 5000 * point_value_per_point
        else:
            milestone_value_for_next_year += suite_upgrade_value

    # 50박: [5K Points or 2 Suite Upgrade]
    if total_nights_year1 >= 50:
        if milestone_50_choice == "5K Points":
            milestone_value_for_next_year += 5000 * point_value_per_point
        else:
            milestone_value_for_next_year += 2 * suite_upgrade_value

    # 60박: 2 GOH + Cat1–7 FN + 2 Suite Upgrade
    if total_nights_year1 >= 60:
        milestone_value_for_next_year += 2 * goh_value
        milestone_value_for_next_year += cat17_value
        milestone_value_for_next_year += 2 * suite_upgrade_value

    # (5) 매트리스런 지불금에 대한 포인트 적립 가치
    #     = 달러당 base_points_per_dollar * (1 + 보너스%) → 포인트 × point_value
    scenario_b_points = (
        scenario_b_mattress_run_cost
        * base_points_per_dollar
        * (1 + globalist_bonus_percent / 100)
    )
    scenario_b_points_value = scenario_b_points * point_value_per_point

    # (6) 내년 숙박(Globalist) 혜택
    #     = (next_year_organic_nights × next_year_globalist_per_night_value)
    #     단, 올해 60박 달성 못하면 0
    if total_nights_year1 >= 60:
        scenario_b_globalist_value_next_year = next_year_organic_nights * next_year_globalist_per_night_value
    else:
        # 60박 미달 → 내년 Globalist 혜택X
        scenario_b_globalist_value_next_year = 0.0
        milestone_value_for_next_year = 0.0  # 리워드도 없음

    # (7) 시나리오 B: 내년 총 혜택 = (Globalist 숙박 혜택 + 마일스톤) + 매트런 포인트 가치
    #     여기서 매트런 포인트 가치는 엄밀히 말하면 "올해" 획득분이지만, 전체 비용을 상쇄하므로 함께 계산
    scenario_b_total_value_next_year = scenario_b_globalist_value_next_year + milestone_value_for_next_year + scenario_b_points_value

    # (8) 시나리오 B 순 가치(넷)
    scenario_b_net = scenario_b_total_value_next_year - scenario_b_mattress_run_cost

    return {
        "Scenario_A": {
            "Next_Year_Benefit": scenario_a_additional_value_next_year,
            "Additional_Cost": scenario_a_additional_cost,
            "Net": scenario_a_net
        },
        "Scenario_B": {
            "Nights_Needed": nights_needed,
            "Mattress_Run_Cost": scenario_b_mattress_run_cost,
            "Next_Year_Benefit_Globalist": scenario_b_globalist_value_next_year,
            "Next_Year_Benefit_Milestone": milestone_value_for_next_year,
            "Value_of_MR_Points": scenario_b_points_value,
            "Total_Value_Next_Year": scenario_b_total_value_next_year,
            "Net": scenario_b_net
        }
    }

##############################################################################
# 출력 함수
##############################################################################

def display_calculation_for_next_year(
    # 올해
    this_year_organic_nights,
    required_nights_for_globalist,
    mattress_run_cost_per_night,

    # 내년
    next_year_organic_nights,
    next_year_globalist_per_night_value,

    # 포인트
    base_points_per_dollar,
    globalist_bonus_percent,
    point_value_per_point,

    # 마일스톤 아이템(1개) 가치
    cat14_value,
    cat17_value,
    goh_value,
    club_access_value,
    suite_upgrade_value,

    # 마일스톤 선택
    milestone_30_choice,
    milestone_40_choice,
    milestone_50_choice
):
    result = globalist_value_next_year(
        this_year_organic_nights=this_year_organic_nights,
        required_nights_for_globalist=required_nights_for_globalist,
        mattress_run_cost_per_night=mattress_run_cost_per_night,

        next_year_organic_nights=next_year_organic_nights,
        next_year_globalist_per_night_value=next_year_globalist_per_night_value,

        base_points_per_dollar=base_points_per_dollar,
        globalist_bonus_percent=globalist_bonus_percent,
        point_value_per_point=point_value_per_point,

        cat14_value=cat14_value,
        cat17_value=cat17_value,
        goh_value=goh_value,
        club_access_value=club_access_value,
        suite_upgrade_value=suite_upgrade_value,

        milestone_30_choice=milestone_30_choice,
        milestone_40_choice=milestone_40_choice,
        milestone_50_choice=milestone_50_choice
    )

    sA = result["Scenario_A"]
    sB = result["Scenario_B"]

    md_text = f"""
### 올해
- 매트리스런 박 수: **{sB['Nights_Needed']}박**
- 매트리스런 비용: **${sB['Mattress_Run_Cost']:.2f}**

- 매트리스런 포인트 적립 가치(올해 획득분): **${sB['Value_of_MR_Points']:.2f}**

**내년**
- Globalist 숙박 혜택: **${sB['Next_Year_Benefit_Globalist']:.2f}**

- 마일스톤 리워드 가치(내년에 사용): **${sB['Next_Year_Benefit_Milestone']:.2f}**


- 내년 총 혜택: **${sB['Total_Value_Next_Year']:.2f}**

- **순 가치(넷) = 내년 총 혜택 - 매트리스런 비용**
  → **${sB['Net']:.2f}**
    """.strip()

    display(Markdown(md_text))

##############################################################################
# 위젯 구성
##############################################################################

# 1) 올해
this_year_nights_input = widgets.IntText(
    value=25, description="올해 실제 숙박(박)", style={'description_width': '130px'}
)

required_nights_input = widgets.IntText(
    value=60, description="Globalist 필요박", style={'description_width': '110px'}
)

mattress_run_cost_input = widgets.FloatText(
    value=80.0, description="매트런 1박($)", style={'description_width': '90px'}
)

# 2) 내년
next_year_nights_input = widgets.IntText(
    value=25, description="내년 실제 숙박(박)", style={'description_width': '130px'}
)
next_year_globalist_value_input = widgets.FloatText(
    value=100.0, description="내년 Globalist 1박($)", style={'description_width': '140px'}
)

# 3) 포인트 적립
base_points_input = widgets.FloatText(
    value=5.0, description="BaseP/$", style={'description_width': '60px'},
    layout=widgets.Layout(width='130px')
)
globalist_bonus_input = widgets.FloatText(
    value=30.0, description="보너스(%)", style={'description_width': '70px'},
    layout=widgets.Layout(width='130px')
)
point_value_input = widgets.FloatText(
    value=0.02, description="1P($)", style={'description_width': '40px'},
    layout=widgets.Layout(width='100px')
)

# 4) 마일스톤 가치(1개)
cat14_value_input = widgets.FloatText(
    value=150.0, description="Cat1–4 FN($)", style={'description_width': '120px'}
)
cat17_value_input = widgets.FloatText(
    value=300.0, description="Cat1–7 FN($)", style={'description_width': '120px'}
)
goh_value_input = widgets.FloatText(
    value=100.0, description="GOH($)", style={'description_width': '40px'}
)
club_value_input = widgets.FloatText(
    value=20.0, description="Club($)", style={'description_width': '40px'}
)
suite_upgrade_input = widgets.FloatText(
    value=150.0, description="SuiteUp($)", style={'description_width': '60px'}
)

# 5) 마일스톤 선택(30/40/50박)
milestone_30_dropdown = widgets.Dropdown(
    options=["2K Points", "2 Club Access"],
    value="2K Points", description="30박:"
)
milestone_40_dropdown = widgets.Dropdown(
    options=["5K Points", "1 Suite Upgrade"],
    value="1 Suite Upgrade",  # 기본 SUA로
    description="40박:"
)
milestone_50_dropdown = widgets.Dropdown(
    options=["5K Points", "2 Suite Upgrade"],
    value="2 Suite Upgrade",  # 기본 2SUA로
    description="50박:"
)

# 레이아웃
row_year1 = widgets.HBox([
    this_year_nights_input,
    required_nights_input,
    mattress_run_cost_input
])
row_year2 = widgets.HBox([
    next_year_nights_input,
    next_year_globalist_value_input
])
row_points = widgets.HBox([
    base_points_input,
    globalist_bonus_input,
    point_value_input
])
row_milestone_values1 = widgets.HBox([
    cat14_value_input,
    cat17_value_input,
])
row_milestone_values2 = widgets.HBox([
    goh_value_input,
    club_value_input,
    suite_upgrade_input
])
row_milestone_choices = widgets.HBox([
    milestone_30_dropdown,
    milestone_40_dropdown,
    milestone_50_dropdown
])

ui_layout = widgets.VBox([
    widgets.Label("=== 올해 매트리스런 설정 ==="),
    row_year1,

    widgets.Label("=== 내년 숙박 설정 ==="),
    row_year2,

    widgets.Label("=== 매트리스런 포인트 적립 ==="),
    row_points,

    widgets.Label("=== 마일스톤 아이템(1개) 가치 ==="),
    row_milestone_values1,
    row_milestone_values2,

    widgets.Label("=== 마일스톤 선택(30/40/50박) ==="),
    row_milestone_choices
])

out = widgets.interactive_output(
    display_calculation_for_next_year,
    {
      # 올해
      'this_year_organic_nights': this_year_nights_input,
      'required_nights_for_globalist': required_nights_input,
      'mattress_run_cost_per_night': mattress_run_cost_input,

      # 내년
      'next_year_organic_nights': next_year_nights_input,
      'next_year_globalist_per_night_value': next_year_globalist_value_input,

      # 포인트 적립
      'base_points_per_dollar': base_points_input,
      'globalist_bonus_percent': globalist_bonus_input,
      'point_value_per_point': point_value_input,

      # 마일스톤 아이템(1개) 가치
      'cat14_value': cat14_value_input,
      'cat17_value': cat17_value_input,
      'goh_value': goh_value_input,
      'club_access_value': club_value_input,
      'suite_upgrade_value': suite_upgrade_input,

      # 마일스톤 선택
      'milestone_30_choice': milestone_30_dropdown,
      'milestone_40_choice': milestone_40_dropdown,
      'milestone_50_choice': milestone_50_dropdown
    }
)

display(ui_layout, out)